In [ ]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import folium
import folium.plugins
import logging
from eo_tools.util import explore_products
from eo_tools.S2 import process_s2_tiles

from pprint import pprint
import xmltodict
import rasterio

from datetime import datetime
from dateutil import parser

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

# change your data directories
path_data = "/data"
path_s2 = f"{path_data}/S2"
path_processed = f"{path_data}/res"

# Search

In [ ]:
# load a geometry
file_aoi = "../data/Bretagne_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S2_MSI_L1C",
    "start": "2023-06-01",
    "end": "2023-08-31",
    "cloudCover": 10,
    "geom": shp
}
results = dag.search_all(**search_criteria)

Getting unique dates to help with product exploration

In [ ]:
def get_unique_dates(results):
    import numpy as np

    dates = []
    for p in results:
        dates.append(p.properties["startTimeFromAscendingNode"])
    return np.unique(dates).tolist()

dates = get_unique_dates(results)
print(f"unique dates:")
print(", \n".join(dates))

# Visualize on a map

In [ ]:
# show all results
# explore_products(results, shp)

# filter by date
results_filtered = results.filter_date("2023-06-05", "2023-06-06") 
# explore_products(results_filtered[1:], shp)

# Download

In [ ]:
# re-run to get only links
dl = dag.download_all(results_filtered[1:], outputs_prefix=path_s2)

# Open tile metadata

In [ ]:
with rasterio.open(dl[0]) as ds:
    # print(ds.tag_namespaces())
    xmlstr = ds.tags(ns='xml:SENTINEL2')['xml:SENTINEL2']
    metadict = xmltodict.parse(xmlstr)
    prof = ds.profile
    tags = ds.tags()
    # meta = 

In [ ]:
proc_bsl = tags["PROCESSING_BASELINE"]
proc_level = tags["PROCESSING_LEVEL"]
QV = tags["QUANTIFICATION_VALUE"]

In [ ]:
pprint(tags)

In [ ]:
# pprint(metadict)
# pprint(metadict['n1:Level-1C_User_Product'].keys())
PIC = metadict["n1:Level-1C_User_Product"]["n1:General_Info"][
    "Product_Image_Characteristics"
]
QV = PIC["QUANTIFICATION_VALUE"]
ROL = PIC["Radiometric_Offset_List"]["RADIO_ADD_OFFSET"]
SIL = PIC["Spectral_Information_List"]["Spectral_Information"]

In [ ]:
float(ROL[0]["#text"])

In [ ]:
for it in SIL:
    print(it["@bandId"], it["@physicalBand"])

In [ ]:
ROL['RADIO_ADD_OFFSET']

In [ ]:
start_time = tags["DATATAKE_1_DATATAKE_SENSING_START"]

In [ ]:
dt_start = parser.parse(start_time)
dt_start.strftime("%Y-%m-%d-%H%M%S")

# Merge downloaded tiles

In [ ]:
out_dirs = process_s2_tiles(
    dl,
    bands=["B2", "B3", "B4"], #, "B10", "B11"],
    shp=shp,
    aoi_name="aoi_bretagne",
    outputs_prefix=path_processed,
)

In [ ]:
from eo_tools.util import visualize_s2_rgb
visualize_s2_rgb(out_dirs[0])

In [ ]:
f"L{proc_level[-2:]}"